<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="../media/inria-25-alpha.png" /></span>
</div>

# quand ça coince

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## git est conservatoire

* plusieurs situations où git va refuser de travailler  
* de bon sens, à chaque fois qu'il y a un risque de perdre des données  
* le plus souvent relatif à des changements  
  qui ne sont **pas encore dans un commit**  

* voyons quelques cas typiques (liste non exhaustive)

## git pull avec modifications pendantes

**le cas le plus fréquent :**  

* vous avez un changement pendant (dans l'index ou pas)  
  dans le fichier `bidule`
  
* vous faites `git pull` pour vous mettre à jour  
  par rapport à *upstream* 
  
* sauf que ce pull **induit aussi** 
  un changement dans `bidule` 
  
dans ce cas d'usage assez fréquent, `git pull` **échoue**


récap  

* changement pendant dans le fichier `bidule`
* tentative de `git pull`
* induit un changement dans `bidule`

il faut donc faire, en ce qui concerne `bidule`  
un *merge* entre  

* le fichier local **qui n'est dans aucun commit**
* le commit qui vient d'upstream

si cela devait aboutir à un conflit,  
on **ne pourrait pas revenir en arrière** 

## mise en place

In [ ]:
# si nécessaire, vous pouvez remettre le repository en l'état
# 
# pour cela mettez "true" au lieu de ""
# et bien sûr évaluer la cellule

reset=""

if [ -n "$reset" ]; then 
    cd $TOP
    bash $SCRIPTS/3-01-my-first-remote.sh
fi >& /dev/null

In [ ]:
# si nécessaire, on se place dans le dépôt git
[ -d repo-alice ] && cd repo-alice

pwd

## vérifions

In [ ]:
# nous sommes dans 'repo-alice' qui a 4 commits

git l

In [ ]:
# et pour être sûr on vérifie aussi 
# qu'il n'y a pas de différence entre
# le commit et les fichiers
# on doit voir 'working tree clean'

git status

## illustration

**scénario**

* je clone `repo-alice` dans `repo-upstream`
* dans le repo *upstream*, je **crée un commit** qui change **le début** de `factorial.py`
* dans `repo-alice` je change cette fois **la fin** de `factorial.py`  
  mais **sans faire de commit**

* toujours dans `repo-alice`, j'**essaie de tirer** le repo *upstream* 


j'observe que `git pull` refuse de travailler  


### on crée un repo *upstream* 

In [ ]:
# je crée le clone

cd $TOP
rm -rf repo-upstream
git clone repo-alice repo-upstream
cd repo-upstream
git l

In [ ]:
# je change le début du fichier python


$SCRIPTS/do change-factorial-py-beginning

git diff

In [ ]:
# je crée le commit correspondant

git add factorial.py
git commit -m "modif début de factorial.py depuis upstream"

### une modification pendante dans `repo-alice`

In [ ]:
cd $TOP/repo-alice
$SCRIPTS/do change-factorial-py-end
git diff

### on ne peut pas tirer !

In [ ]:
cd $TOP/repo-alice
git pull ../repo-upstream

### git est (très) conservatoire

* nous avons fait attention à faire deux changements  
  qui sont **non conflictuels**

* du coup on pourrait s'attendre à ce que `git pull` parvienne  
  à s'en sortir

* et pourtant git **refuse de seulement essayer**
* à cause du risque de perte de pertes
* car **on n'a plus nulle part la version de `factorial.py`
  avec seulement les changements locaux !

## deux options pour s'en sortir

dans ce cas - relativement fréquent -  
vous avez deux options pour vous en sortir :

* commencez par committer vos propres changements,  
  car là on a une sauvegarde et git acceptera   
  de prendre le risque de conflit  

* ou créez un *stash*


## git checkout avec modifications pendantes

le symptôme est **exactement le même**  
si vous essayez de **changer de branche** (avec git `checkout`)  
avec des **modifications pendantes** sur des fichiers  
auxquels le `git checkout` aurait **besoin de toucher**  

(puisque, on le rappelle, changer de branche induit de maintenir votre espace de travail en cohérence avec la nouvelle branche)

même symptôme, et mêmes remèdes (commit ou stash)

## git stash

l'idée générale du *stash* (cachette en anglais) :

* enlever les modifications pendantes
* les ranger à la place dans un espace dédié
* que l'on peut réutiliser ensuite
* typiquement pour réappliquer les changements ultérieurement

## git pull grâce à un stash

nous allons résoudre notre problème initial avec un stash  

In [ ]:
cd $TOP/repo-alice
git diff

In [ ]:
# dans un premier temps on range nos modifications courantes
# dans un objet de type stash

# on lui donne un message pour le retrouver
# car on peut créer autant de stashes que l'on veut
git stash push -m"change circa the end of factorial.py"

In [ ]:
# le point c'est que maintenant notre dépôt est propre
git status

In [ ]:
# si on veut on peut lister les stashes 

git stash list

In [ ]:
# comme le repo est propre maintenant
# on peut tirer sans souci

git pull ../repo-upstream

In [ ]:
# et pour remettre les changements locaux
# dans nos fichiers, on applique le stash 
# avec la commande `git stash pop`
git stash pop 'stash@{0}'

In [ ]:
# on est bien dans l'état où on voulait être
# le commit de upstream a été tiré
# et on a préservé la modification locale 
git l

In [ ]:
git diff

## résumé

* `git stash` permet de "mettre de côté"  
  des modifications pendantes

* de manière à revenir à un repo propre
* pour pouvoir faire `pull` ou `checkout` (entre autres)
* quand on a un dépôt encombré de modifications
* que l'on ne souhaite pas forcément committer
